In [ ]:
import pandas as pd
import numpy as np
import faiss  
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from tqdm import tqdm
from collections import Counter

#Carregamento e Limpeza 

In [ ]:
csv_files = [
    "novods/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv",
    "novods/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv",
    "novods/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv",
    "novods/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv",
    "novods/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv",
    #caso queira carregar os outros arquivos, só descomentar,
    #mas vai precisar alterar os testes para refletir a realidade dos ataques majoritarios e minoritarios
    #"novods/Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv", s
    #"novods/Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv",
    #"novods/Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv",
    #"novods/Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv",
]

dataframes_list = []
for file_path in csv_files: #ler os arquivos
    df_temp = pd.read_csv(file_path, low_memory=False) #salvar em um dataframe temporario
    df_temp.drop_duplicates(inplace=True) #dropar as duplicatas
    dataframes_list.append(df_temp) #lista dos dataframes

print("Concatenando datasets...") #concatenar os datasets
dataset_ids = pd.concat(dataframes_list, ignore_index=True) #
dataset_ids.columns = dataset_ids.columns.str.strip() 

# Seleção de Features
feature_columns = [
    "Label", "Protocol", "Fwd Header Len", "Bwd Header Len", "Flow IAT Mean",
    "Idle Mean", "Bwd Pkt Len Mean", "Fwd Pkt Len Mean", "Bwd IAT Mean", "Fwd IAT Mean",
    "Active Mean", "SYN Flag Cnt", "PSH Flag Cnt", "ACK Flag Cnt", "URG Flag Cnt",
    "FIN Flag Cnt", "Fwd Pkt Len Min", "Flow IAT Min", "Pkt Len Min", "Fwd Pkt Len Max",
    "Flow IAT Max", "Pkt Len Max", "Active Max", "Active Min"
]

features = dataset_ids[feature_columns].copy()

# Criação do Flow_Label (ID único do fluxo)
columns_to_concat = [col for col in feature_columns if col not in ['Label', 'Protocol']]
features["Flow_Label"] = features[columns_to_concat].astype(str).agg('-'.join, axis=1)
features["Flow_Label"] = features["Flow_Label"].str.lower().str.strip()

features = features.drop_duplicates(subset=['Flow_Label'])
features = features.set_index("Flow_Label")

# Separar X (features numéricas) e y (Rótulo)
X_raw = features.drop(columns=['Label', 'Protocol'], errors='ignore')
y_raw = features['Label']

print("Sanitizando dados numéricos (removendo cabeçalhos repetidos e strings inválidas)...")
# converte tudo pra numerico, pra nao ter uma string inesperada la no meio
X_raw = X_raw.apply(pd.to_numeric, errors='coerce')

# limpa as linhas com NaN
rows_with_nan = X_raw.isnull().any(axis=1)
if rows_with_nan.sum() > 0:
    print(f"   AVISO: Removendo {rows_with_nan.sum()} linhas inválidas que não puderam ser convertidas para float.")
    # Remove essas linhas tanto do X quanto do y para manter a consistência
    X_raw = X_raw[~rows_with_nan]
    y_raw = y_raw[~rows_with_nan]

print("Dados limpos.")

# descricao do dataset (numero de entradas por label)
print("\n" + "="*40)
print("       RESUMO DO DATASET      ")
print("="*40)
print(f"Shape Total: {X_raw.shape} (Linhas, Colunas)")
print("\nDistribuição de Rótulos (Top 20):")
print(y_raw.value_counts().head(20))
print("\nInformações das Colunas:")
print(X_raw.info(verbose=False))
print("="*40 + "\n")

Concatenando datasets...
Sanitizando dados numéricos (removendo cabeçalhos repetidos e strings inválidas)...
   AVISO: Removendo 1 linhas inválidas que não puderam ser convertidas para float.
Dados limpos.

       RESUMO DO DATASET      
Shape Total: (2989048, 22) (Linhas, Colunas)

Distribuição de Rótulos (Top 20):
Label
Benign                      2596445
DoS attacks-Hulk             145009
Bot                          143967
Infilteration                 51834
DoS attacks-GoldenEye         41356
DoS attacks-Slowloris          9859
Brute Force -Web                337
Brute Force -XSS                143
DoS attacks-SlowHTTPTest         55
SQL Injection                    43
Name: count, dtype: int64

Informações das Colunas:
<class 'pandas.core.frame.DataFrame'>
Index: 2989048 entries, 192-152-9425.666667-0.0-539.0-61.44444444-8569.5-17673.125-0.0-0-1-0-0-0-0-1.0-0-202-73403.0-1460-0.0-0.0 to 372-352-3436975.32352941-58082282.0-309.7058823529-59.2222222222-7288026.0625-6873950.6470588

#Set-up do FAISS

In [10]:
class FAISS_IDS_Recommender:
    def __init__(self, k_neighbors=50):
        self.k = k_neighbors
        self.index = None
        self.train_labels = None
        self.scaler = StandardScaler()
    
    def fit(self, X_train, y_train):
        print(f"   [FAISS] Iniciando fit com {len(X_train)} amostras...")
        if len(X_train) == 0:
            raise ValueError("conjunto de treino vazio, revisar divisao dos dados")

        # 1. Scaling
        X_scaled = self.scaler.fit_transform(X_train).astype('float32')
        

        #garantir c-contiguous array pro faiss (se nao ele quebra)
        X_scaled = np.ascontiguousarray(X_scaled)

        #normalizacao l2
        faiss.normalize_L2(X_scaled)
        
        # 3. Criação do Índice FAISS
        d = X_scaled.shape[1]
        self.index = faiss.IndexFlatIP(d)
        self.index.add(X_scaled)
        
        # 4. Lookup Array
        self.train_labels = y_train.to_numpy()
        print("   [FAISS] Índice construído.")
        
    def predict(self, X_test):
        print(f"   [FAISS] Iniciando busca para {len(X_test)} amostras de teste...")
        if len(X_test) == 0:
            return []

        # 1. Preprocessamento do Teste
        X_test_scaled = self.scaler.transform(X_test).astype('float32')

        # garantir c-contiguous array pro faiss (se nao ele quebra)
        X_test_scaled = np.ascontiguousarray(X_test_scaled)

        #normalizacao l2
        faiss.normalize_L2(X_test_scaled)
        
        # se k for maior doq o numero de amostras, reduz k
        k_search = min(self.k, len(self.train_labels))

        D, I = self.index.search(X_test_scaled, k_search)
        
        # 3. Atribuição de Rótulos
        neighbors_labels = self.train_labels[I]
        
        # 4. Votação
        classified_attacks = []
        for row_labels in tqdm(neighbors_labels, desc="   [FAISS] Classificando"):
            counts = Counter(row_labels)
            # Retorna lista ordenada por score
            sorted_attacks = [lbl for lbl, cnt in counts.most_common()]
            classified_attacks.append(sorted_attacks)
            
        return classified_attacks


#função para executar os cenários

In [11]:
def run_test_scenario(test_name, test_size_map, full_data_X, full_data_y):
    print(f"\n{'='*60}\nIniciando {test_name}\n{'='*60}")
    
    train_idxs = []
    test_idxs = []
    
    # --- divisao de dados --- 
    unique_labels = full_data_y.unique()
    
    for label in unique_labels:
        # para cada classe, pega os indices dela
        indices = full_data_y[full_data_y == label].index 
        
        if len(indices) < 2:
            # se só tem 1 amostra, vai pro treino (sanity test)
            train_idxs.extend(indices)
            continue
            
        test_size = test_size_map.get(label, 0.3)
        
        if test_size >= 1.0:
            # 100% Teste, 0% Treino
            test_idxs.extend(indices)
            continue
            
        if test_size <= 0.0:
            # 0% Teste, 100% Treino
            train_idxs.extend(indices)
            continue
            
        try:
            train_i, test_i = train_test_split(indices, test_size=test_size, random_state=42)
            train_idxs.extend(train_i)
            test_idxs.extend(test_i)
        except ValueError:
            train_idxs.extend(indices)

    # cria os dataframes de treino e teste com os indices que pegamos no passo anterior
    X_train = full_data_X.loc[train_idxs]
    y_train = full_data_y.loc[train_idxs]
    
    X_test = full_data_X.loc[test_idxs]
    y_test = full_data_y.loc[test_idxs]

    print(f"Tamanho Treino: {len(X_train)} | Tamanho Teste: {len(X_test)}")
    
    # --- executa o faiss ---
    recommender = FAISS_IDS_Recommender(k_neighbors=50)
    recommender.fit(X_train, y_train)
    y_pred_lists = recommender.predict(X_test)
    
    # --- avaliação ---
    # y_test convertido para lista de listas
    y_true_lists = [[label] for label in y_test]
    
    
    # coletar todas as classes possíveis
    all_classes = sorted(list(set(y_train.unique()) | set(y_test.unique())))
    mlb = MultiLabelBinarizer(classes=all_classes)
    
    #top 1 pred vs real label
    y_pred_top1 = [[preds[0]] if preds else [] for preds in y_pred_lists]
    
    y_true_bin = mlb.fit_transform(y_true_lists)
    y_pred_bin = mlb.transform(y_pred_top1) # Avaliando Top-1
    
    print(f"\n--- Relatório de Classificação (Top-1 Recommendation) para {test_name} ---")
    report = classification_report(y_true_bin, y_pred_bin, target_names=mlb.classes_, zero_division=0)
    print(report)


#Definição e execução dos testes

In [12]:
# CENÁRIO 1: (70% Treino / 30% Teste)
# Todos as classes divididas igual
map_1 = {label: 0.3 for label in y_raw.unique()}
run_test_scenario("Teste 1: 70% Treino / 30% Teste)", map_1, X_raw, y_raw)

# CENÁRIO 2: Ataques Menos Comuns
# selecionados os 5 ataques menos frequentes 70% treino 30% teste"
minority_attacks = [
    'Brute Force -Web', 'Brute Force -XSS', 'DoS attacks-SlowHTTPTest', 
    'DoS attacks-Slowloris', 'SQL Injection'
]

# Filtrando dados
mask_minority = y_raw.isin(minority_attacks)
X_minor = X_raw[mask_minority]
y_minor = y_raw[mask_minority]
map_2 = {label: 0.3 for label in y_minor.unique()}

run_test_scenario("Teste 2: Ataques Menos Comuns", map_2, X_minor, y_minor)


# CENÁRIO 3: Teste 2 do Artigo (Treino Desbalanceado, Teste Total)
# "Treino composto exclusivamente por amostras das 3 classes com maior número (+ benign)... 
# divisão de 25% para treino (desses 4) e o resto teste"
major_attacks = ['DoS attacks-Hulk', 'Infilteration', 'Bot', 'Benign'] 
map_3 = {}
for label in y_raw.unique():
    if label in major_attacks:
        map_3[label] = 0.75 # 25% Treino, 75% Teste
    else:
        map_3[label] = 1.0 # 0% Treino (100% Teste) - Simula que não conhece os outros

run_test_scenario("Teste 3: Treino de 25% das classes majoritárias, Teste com todo o dataset", map_3, X_raw, y_raw)


# CENÁRIO 4: Teste 3 do Artigo (10% Treino, 90% Teste Global)
map_4 = {label: 0.90 for label in y_raw.unique()} # 10% Treino, 90% Teste
run_test_scenario("Teste 4: 10% Treino / 90% Teste (Todos os Dados)", map_4, X_raw, y_raw)


Iniciando Teste 1: 70% Treino / 30% Teste)
Tamanho Treino: 2092329 | Tamanho Teste: 896719
   [FAISS] Iniciando fit com 2092329 amostras...
   [FAISS] Índice construído.
   [FAISS] Iniciando busca para 896719 amostras de teste...


   [FAISS] Classificando: 100%|██████████| 896719/896719 [00:03<00:00, 297900.41it/s]



--- Relatório de Classificação (Top-1 Recommendation) para Teste 1: 70% Treino / 30% Teste) ---
                          precision    recall  f1-score   support

                  Benign       0.98      1.00      0.99    778934
                     Bot       1.00      0.99      1.00     43191
        Brute Force -Web       0.92      0.24      0.38       102
        Brute Force -XSS       1.00      0.40      0.57        43
   DoS attacks-GoldenEye       0.98      1.00      0.99     12407
        DoS attacks-Hulk       1.00      1.00      1.00     43503
DoS attacks-SlowHTTPTest       0.71      1.00      0.83        17
   DoS attacks-Slowloris       0.96      0.99      0.98      2958
           Infilteration       0.84      0.10      0.17     15551
           SQL Injection       0.00      0.00      0.00        13

               micro avg       0.98      0.98      0.98    896719
               macro avg       0.84      0.67      0.69    896719
            weighted avg       0.98      0.

   [FAISS] Classificando: 100%|██████████| 3133/3133 [00:00<00:00, 237588.00it/s]


--- Relatório de Classificação (Top-1 Recommendation) para Teste 2: Ataques Menos Comuns ---
                          precision    recall  f1-score   support

        Brute Force -Web       0.80      0.93      0.86       102
        Brute Force -XSS       0.94      0.67      0.78        43
DoS attacks-SlowHTTPTest       1.00      1.00      1.00        17
   DoS attacks-Slowloris       1.00      1.00      1.00      2958
           SQL Injection       0.00      0.00      0.00        13

               micro avg       0.99      0.99      0.99      3133
               macro avg       0.75      0.72      0.73      3133
            weighted avg       0.99      0.99      0.99      3133
             samples avg       0.99      0.99      0.99      3133


Iniciando Teste 3: Treino de 25% das classes majoritárias, Teste com todo o dataset


Tamanho Treino: 734312 | Tamanho Teste: 2254736
   [FAISS] Iniciando fit com 734312 amostras...
   [FAISS] Índice construído.
   [FAISS] Iniciando busca para 2254736 amostras de teste...


   [FAISS] Classificando: 100%|██████████| 2254736/2254736 [00:08<00:00, 258854.96it/s]



--- Relatório de Classificação (Top-1 Recommendation) para Teste 3: Treino de 25% das classes majoritárias, Teste com todo o dataset ---
                          precision    recall  f1-score   support

                  Benign       0.96      1.00      0.98   1947334
                     Bot       1.00      0.99      1.00    107976
        Brute Force -Web       0.00      0.00      0.00       337
        Brute Force -XSS       0.00      0.00      0.00       143
   DoS attacks-GoldenEye       0.00      0.00      0.00     41356
        DoS attacks-Hulk       0.96      1.00      0.98    108757
DoS attacks-SlowHTTPTest       0.00      0.00      0.00        55
   DoS attacks-Slowloris       0.00      0.00      0.00      9859
           Infilteration       0.81      0.08      0.15     38876
           SQL Injection       0.00      0.00      0.00        43

               micro avg       0.96      0.96      0.96   2254736
               macro avg       0.37      0.31      0.31   2254736
  

   [FAISS] Classificando: 100%|██████████| 2690149/2690149 [00:09<00:00, 280061.63it/s]



--- Relatório de Classificação (Top-1 Recommendation) para Teste 4: 10% Treino / 90% Teste (Todos os Dados) ---
                          precision    recall  f1-score   support

                  Benign       0.98      1.00      0.99   2336801
                     Bot       1.00      0.99      0.99    129571
        Brute Force -Web       0.00      0.00      0.00       304
        Brute Force -XSS       0.00      0.00      0.00       129
   DoS attacks-GoldenEye       0.94      0.97      0.96     37221
        DoS attacks-Hulk       0.99      1.00      0.99    130509
DoS attacks-SlowHTTPTest       0.00      0.00      0.00        50
   DoS attacks-Slowloris       0.94      0.93      0.93      8874
           Infilteration       0.78      0.06      0.11     46651
           SQL Injection       0.00      0.00      0.00        39

               micro avg       0.98      0.98      0.98   2690149
               macro avg       0.56      0.49      0.50   2690149
            weighted avg   